In [1]:
#!pip install textattack

In [2]:
import numpy as np
import pandas as pd

from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

In [3]:
!gpustat

seven                Thu Mar  4 14:04:33 2021  440.64
[0] TITAN RTX        | 86'C,  99 % | 17134 / 24220 MB | sharuev(1649M) sharuev(2549M) sharuev(12925M)
[1] TITAN RTX        | 88'C,  99 % | 10230 / 24220 MB | cwb(3297M) cwb(3955M) cwb(2967M)
[2] TITAN RTX        | 47'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 88'C,  98 % |  9156 / 24220 MB | cwb(2917M) cwb(3053M) cwb(3175M)


In [4]:
device = 'cuda:0'

In [5]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [6]:
class GenreDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
def load_dataset_from_csv(filename):
    df = pd.read_csv(filename).sample(frac=1, random_state=42)
    df = df[df.target != 'A22']
    unique_labels = np.unique(sorted(df.target.values))
    label_to_id = {label: label_id for label_id, label in enumerate(unique_labels)}
    return list(df.text.values), [label_to_id[label] for label in df.target.values]

In [8]:
train_texts, train_labels = load_dataset_from_csv('/home/mlepekhin/data/ru_train')
test_texts, test_labels = load_dataset_from_csv('/home/mlepekhin/data/ru_test')

In [9]:
import pickle

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
#train_encodings = pickle.load(open('ru_train_xlm_roberta_encodings.pcl', 'rb'))
#test_encodings = pickle.load(open('ru_test_xlm_roberta_encodings.pcl', 'rb'))

In [10]:
train_dataset = GenreDataset(train_encodings, train_labels)
test_dataset = GenreDataset(test_encodings, test_labels)

In [11]:
from torch.utils.data import DataLoader

#train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12)

In [12]:
!mkdir xlm_roberta_ru

mkdir: cannot create directory ‘xlm_roberta_ru’: File exists


In [13]:
from sklearn.metrics import accuracy_score
from transformers import AdamW
from tqdm import tqdm
from collections import defaultdict

In [14]:
for val_step in range(5):
    model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=10).to(device)
    optim = AdamW(model.parameters(), lr=5e-5)
    total_test_accuracy = 0.0
    
    train_subset = defaultdict(list)
    train_subset_labels = []
    val_subset = defaultdict(list)
    val_labels = []
    
    for i in range(len(train_dataset)):
        if i >= val_step * 0.2 * len(train_dataset) and i < (val_step+1) * 0.2 * len(train_dataset):
            val_subset['input_ids'].append(train_encodings['input_ids'][i])
            val_subset['attention_mask'].append(train_encodings['attention_mask'][i])
            val_labels.append(train_labels[i])
        else:
            train_subset['input_ids'].append(train_encodings['input_ids'][i])
            train_subset['attention_mask'].append(train_encodings['attention_mask'][i])
            train_subset_labels.append(train_labels[i])
    
    train_loader = DataLoader(GenreDataset(train_subset, train_subset_labels), batch_size=8, shuffle=True)
    val_loader = DataLoader(GenreDataset(val_subset, val_labels), batch_size=8)

    for epoch in range(5):
        model.train()
        train_accuracy = 0
        item_count = 0

        for batch in tqdm(train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            item_count += batch['labels'].shape[0]
            #start_positions = batch['start_positions'].to(device)
            #end_positions = batch['end_positions'].to(device)
            output = model(input_ids, attention_mask=attention_mask, labels=labels)
            cur_accuracy = np.sum(batch['labels'].numpy() == np.argmax(output.logits.cpu().detach().numpy(), axis=1))
            train_accuracy += cur_accuracy 
            loss = output.loss
            loss.backward()
            optim.step()
            print(f'loss: {loss.item()}, accuracy: {train_accuracy / item_count}')
            
        model.eval()
        all_outputs = [] 

        for batch in tqdm(val_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            all_outputs.append(np.argmax(outputs.cpu().detach().numpy(), axis=1))
        all_outputs = np.concatenate(all_outputs)
        val_accuracy = accuracy_score(all_outputs, val_labels)
        print(f"epoch: {epoch} val accuracy: {val_accuracy}")
        
        all_outputs = [] 

        for batch in tqdm(test_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask).logits
            all_outputs.append(np.argmax(outputs.cpu().detach().numpy(), axis=1))
        all_outputs = np.concatenate(all_outputs)
        test_accuracy = accuracy_score(all_outputs, test_labels)

        print(f"epoch: {epoch} test accuracy: {test_accuracy}")
        if test_accuracy > total_test_accuracy:
            total_test_accuracy = test_accuracy
            torch.save(model.state_dict(), f'xlm_roberta_ru/new_state_dict_val_{val_step}.pt')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

loss: 2.3576607704162598, accuracy: 0.0


  1%|▏         | 2/145 [00:00<00:44,  3.19it/s]

loss: 2.3383381366729736, accuracy: 0.1875


  2%|▏         | 3/145 [00:00<00:43,  3.25it/s]

...

loss: 0.05938826873898506, accuracy: 0.8272884283246977


  2%|▏         | 1/41 [00:00<00:05,  7.45it/s]

epoch: 4 val accuracy: 0.7750865051903114


100%|██████████| 41/41 [00:05<00:00,  7.40it/s]

epoch: 4 test accuracy: 0.7101449275362319


In [15]:
!gpustat

seven                Thu Mar  4 14:30:31 2021  440.64
[0] TITAN RTX        | 86'C,  82 % | 17134 / 24220 MB | sharuev(1649M) sharuev(2549M) sharuev(12925M)
[1] TITAN RTX        | 87'C,  98 % | 10230 / 24220 MB | cwb(3297M) cwb(3955M) cwb(2967M)
[2] TITAN RTX        | 75'C,  97 % | 23194 / 24220 MB | mlepekhin(23183M)
[3] TITAN RTX        | 87'C,  98 % |  9156 / 24220 MB | cwb(2917M) cwb(3053M) cwb(3175M)
